In [0]:
from cross_entropy import Cross_Entropy
from lag import LAG
from llsr import LLSR as myLLSR
from pixelhop2 import Pixelhop2
from torchvision import transforms, datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.datasets import cifar10
from skimage.util import view_as_windows
from skimage.measure import block_reduce
from sklearn import datasets
import pickle
from numpy import save
from numpy import load
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb


Using TensorFlow backend.


In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
(training, trainLabel),(testing, testLabel) = cifar10.load_data()

#Remove Before Submitting
#print(training.shape)
#print(trainLabel.shape)
#print(testing.shape)
#print(testLabel.shape)

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [0]:
#Conver the data to float and normalize it to values in between (0-1)
training = (training.astype('float'))
testing = (testing.astype('float'))
training /= 255.0
testing /= 255.0
#Mean-SD normalization
avg = np.mean(training, axis=0)
SD = np.std(training, axis=0)
training -= avg
testing -= avg
training /= SD
testing /= SD

In [0]:
#Create New data for Training PixelHop Unit
X_new_training, X_rest, X_new_label, X_rest_label = train_test_split(training, trainLabel, test_size = 0.8, random_state = 0, stratify = trainLabel)

In [0]:
#Constructing the neighborhood
def Shrink(X_new_training, shrinkArg):
  win = shrinkArg['win']
  stride = shrinkArg['stride']
  ch = X_new_training.shape[-1]
  X_new_training = view_as_windows(X_new_training, (1,win,win,ch), (1, stride, stride,ch))
  X_new_training = X_new_training.reshape(X_new_training.shape[0], X_new_training.shape[1], X_new_training.shape[2], -1)
  X_pool = block_reduce(X_new_training, block_size = (1,2,2,1), func = np.max)
  return X_pool

def Concat(X_new_training, concatArg):
  return X_new_training



shrinkArgs = [{'func':Shrink, 'win':3, 'stride': 1},
                {'func': Shrink, 'win':3, 'stride': 1}, 
                {'func': Shrink, 'win':3, 'stride': 1},
              {'func': Shrink, 'win':3, 'stride': 1}]
              
SaabArgs = [{'num_AC_kernels':-1, 'needBias':False, 'useDC':True, 'batch':None, 'cw': False},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw': True}, 
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw': True}]

concatArg = {'func':Concat}

In [0]:
p2 = Pixelhop2(depth=4, TH1=0.001, TH2=0.0001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
p2.fit(X_new_training)
#pickle.dump(p2, open('drive/My Drive/569_Competition/submission_pixelHop_4units.sav', 'wb'))
# output = p2.transform(data_train)
op1 = p2.transform(training[:10000,:,:])
op2 = p2.transform(training[10000:20000,:,:])
op3 = p2.transform(training[20000:30000,:,:])
op4 = p2.transform(training[30000:40000,:,:])
op5 = p2.transform(training[40000:50000,:,:])

pixelhop2 fit
pixelhop2 transform
pixelhop2 transform
pixelhop2 transform
pixelhop2 transform
pixelhop2 transform


In [0]:
ph_1 = np.concatenate((op1[0],op2[0],op3[0],op4[0],op5[0]))
ph_2 = np.concatenate((op1[1],op2[1],op3[1],op4[1],op5[1]))
ph_3 = np.concatenate((op1[2],op2[2],op3[2],op4[2],op5[2]))
ph_4 = np.concatenate((op1[3],op2[3],op3[3],op4[3],op5[3]))
#print(ph_1.shape, ph_2.shape, ph_3.shape,ph_4.shape )
# print(output[0].shape, output[1].shape, output[2].shape)
from numpy import save
save('drive/My Drive/569_Competition/module10.npy', ph_1)
save('drive/My Drive/569_Competition/module11.npy', ph_2)
save('drive/My Drive/569_Competition/module12.npy', ph_3)
save('drive/My Drive/569_Competition/module13.npy', ph_4)


(50000, 15, 15, 22) (50000, 7, 7, 85) (50000, 3, 3, 197) (50000, 1, 1, 385)


(50000, 15, 15, 22) (50000, 7, 7, 85) (50000, 3, 3, 197) (50000, 1, 1, 385)


In [0]:
from numpy import load
ph_1 = load('drive/My Drive/569_Competition/module10.npy')
ph_2 = load('drive/My Drive/569_Competition/module11.npy')
ph_3 = load('drive/My Drive/569_Competition/module12.npy')
ph_4 = load('drive/My Drive/569_Competition/module13.npy')

p2 = pickle.load(open('drive/My Drive/569_Competition/submission_pixelHop_4units.sav', 'rb'))
#m1 = module10.reshape((50000, -1))
print(ph_1.shape)
print(ph_2.shape)
print(ph_3.shape)
print(ph_4.shape)

In [0]:
#Cross Entropy for Module 1
reshape1 = ph_1.reshape((len(ph_1), -1))
CrossEntropy = Cross_Entropy(num_class = 10, num_bin = 5)
CEop1 = np.zeros((reshape1.shape[-1]))

for i in range((reshape1.shape[-1])):
  CEop1[i] = CrossEntropy.KMeans_Cross_Entropy(reshape1[:,i].reshape(-1,1), trainLabel)

input feature shape is: (50000, 4950)
------CE_Module_1 Completed-----



In [0]:
#save('drive/My Drive/569_Competition/Submission_ce_kmeans1_ph4.npy', CEop1)

In [16]:
#Cross Entropy Module 2
reshape2 = ph_2.reshape((len(ph_2), -1)) 
CrossEntropy = Cross_Entropy(num_class = 10, num_bin = 5)
CEop2 = np.zeros((reshape2.shape[-1]))

for i in range((reshape2.shape[-1])):
  CEop2[i] = CrossEntropy.KMeans_Cross_Entropy(reshape2[:,i].reshape(-1,1), trainLabel)

input feature shape is: (50000, 4165)
CE_Module_2 Completed



In [0]:
#save('drive/My Drive/569_Competition/Submission_ce_kmeans2_ph4.npy', CEop2)

In [18]:
#Cross Entropy Module 3
reshape3 = ph_3.reshape((len(ph_3), -1))
CrossEntropy = Cross_Entropy(num_class = 10, num_bin = 5)
CEop3 = np.zeros((reshape3.shape[-1]))

for i in range((reshape3.shape[-1])):
  CEop3[i] = CrossEntropy.KMeans_Cross_Entropy(reshape3[:,i].reshape(-1,1), trainLabel)


input feature shape is: (50000, 1773)
CE_Module_3 Completed



In [0]:
#save('drive/My Drive/569_Competition/Submission_ce_kmeans3_ph4.npy', CEop3)

In [20]:
#Cross Entropy Module 4
reshape4 = ph_4.reshape((len(ph_4), -1))
CrossEntropy = Cross_Entropy(num_class = 10, num_bin = 5)
CEop4 = np.zeros((reshape4.shape[-1]))
for i in range((reshape4.shape[-1])):
  CEop4[i] = CrossEntropy.KMeans_Cross_Entropy(reshape4[:,i].reshape(-1,1), trainLabel)


input feature shape is: (50000, 385)
CE_Module_4 Completed



In [0]:
#save('drive/My Drive/569_Competition/Submission_ce_kmeans4_ph4.npy', CEop4)

In [0]:
#Loading CE Model
#CEop1 = load('drive/My Drive/569_Competition/Submission_ce_kmeans1_ph4.npy')
#CEop2 = load('drive/My Drive/569_Competition/Submission_ce_kmeans1_ph4.npy')
#CEop3 = load('drive/My Drive/569_Competition/Submission_ce_kmeans3_ph4.npy')
#CEop4 = load('drive/My Drive/569_Competition/Submission_ce_kmeans4_ph4.npy')

In [22]:
#Using argsort to sort the cross entropy from lowest to highest
index1 = np.argsort(CEop1)
#Getting the indices of first 1000 values
sorted1 = index1[:1000]
#Selecting the corresponding features
ph_1 = ph_1.reshape(len(ph_1),-1)
m1 = (np.transpose(ph_1))
inlag1 = m1[sorted1]
inlag1 = inlag1.reshape(len(inlag1),-1)
lag1_input=np.transpose(inlag1)
lag1 = LAG(encode='distance', num_clusters = [5,5,5,5,5,5,5,5,5,5], alpha=10, learner=myLLSR(onehot=False))
lag1.fit(lag1_input, trainLabel)
#pickle.dump(lag1, open('drive/My Drive/569_Competition/lag1_model_ph4.sav', 'wb'))


lag1_output = lag1.transform(lag1_input)
lag1_output_prob = lag1.predict_proba(lag1_input)



Input of Lag 1:  (50000, 1000)
Lag1 Output:  (50000, 50)


In [0]:
#save('drive/My Drive/569_Competition/Lag1_output_ph4.npy', lag1_output)

In [25]:
#Using argsort to sort the cross entropy from lowest to highest
index2 = np.argsort(CEop2)
#Getting the indices of first 1000 values
sorted2 = index2[:1000]
#Selecting the corresponding features
ph_2 = ph_2.reshape(len(ph_2),-1)
m2 = (np.transpose(ph_2))
inlag2 = m2[sorted2]
inlag2 = inlag2.reshape(len(inlag2),-1)
lag2_input=np.transpose(inlag2)


lag2 = LAG(encode='distance', num_clusters = [5,5,5,5,5,5,5,5,5,5], alpha=10, learner=myLLSR(onehot=False))
lag2.fit(lag2_input, trainLabel)
#pickle.dump(lag2, open('drive/My Drive/569_Competition/lag2_model_ph4.sav', 'wb'))


lag2_output = lag2.transform(lag2_input)
lag2_output_prob = lag2.predict_proba(lag2_input)

Input of Lag 2:  (50000, 1000)
Lag2 Output:  (50000, 50)


In [0]:
#save('drive/My Drive/569_Competition/Lag2_output_ph4.npy', lag2_output)

In [27]:
#Using argsort to sort the cross entropy from lowest to highest
index3 = np.argsort(CEop3)
#Getting the indices of first 1000 values
sorted3 = index3[:1000]
#Selecting the corresponding features
ph_3 = ph_3.reshape(len(ph_3),-1)
m3 = (np.transpose(ph_3))
inlag3 = m3[sorted3]
inlag3 = inlag3.reshape(len(inlag3),-1)
lag3_input=np.transpose(inlag3)

lag3 = LAG(encode='distance', num_clusters = [5,5,5,5,5,5,5,5,5,5], alpha=10, learner=myLLSR(onehot=False))
lag3.fit(lag3_input, trainLabel)
#pickle.dump(lag3, open('drive/My Drive/569_Competition/lag3_model_ph4.sav', 'wb'))


lag3_output = lag3.transform(lag3_input)
lag3_output_prob = lag3.predict_proba(lag3_input)

Input of Lag 3:  (50000, 1000)
Lag3 Output:  (50000, 50)


In [0]:
#save('drive/My Drive/569_Competition/Lag3_output_ph4.npy', lag3_output)

In [30]:
#Using argsort to sort the cross entropy from lowest to highest
index4 = np.argsort(CEop4)
#Getting the indices of first 1000 values
sorted4 = index4[:1000]
#Selecting the corresponding features
ph_4 = ph_4.reshape(len(ph_4),-1)
m4 = (np.transpose(ph_4))
inlag4 = m4[sorted4]
inlag4 = inlag4.reshape(len(inlag4),-1)
lag4_input=np.transpose(inlag4)


lag4 = LAG(encode='distance', num_clusters = [5,5,5,5,5,5,5,5,5,5], alpha=10, learner=myLLSR(onehot=False))
lag4.fit(lag4_input, trainLabel)
#pickle.dump(lag4, open('drive/My Drive/569_Competition/lag4_model_ph4.sav', 'wb'))


lag4_output = lag4.transform(lag4_input)
lag4_output_prob = lag4.predict_proba(lag4_input)


Input of Lag 4:  (50000, 385)
Lag4 Output:  (50000, 50)


In [32]:
#Prepare data for Classification (Concate LAG outputs)

FeatureMatrix_Train_ = np.concatenate((lag1_output,lag2_output,lag3_output, lag4_output),axis=1)
#save('drive/My Drive/569_Competition/FeatureMatrixForTraining_ph4.npy', FeatureMatrix_Train_)

(50000, 200)


In [33]:
#Prepare test data 
p2Test = p2.transform(testing)
testPH1=p2Test[0].reshape(len(p2Test[0]),-1)
testPH2=p2Test[1].reshape(len(p2Test[1]),-1)
testPH3=p2Test[2].reshape(len(p2Test[2]),-1)
testPH4=p2Test[3].reshape(len(p2Test[3]),-1)


pixelhop2 transform
(10000, 15, 15, 22)
(10000, 7, 7, 85)
(10000, 3, 3, 197)
(10000, 1, 1, 385)
(10000, 4950)
(10000, 4165)
(10000, 1773)
(10000, 385)


In [34]:
#Test data in LAG unit
testPH1=np.transpose(testPH1)
testLAG1 = testPH1[sorted1]
testLAG1 = testLAG1.reshape(len(testLAG1),-1)
testLAG1=np.transpose(testLAG1)

testPH2=np.transpose(testPH2)
testLAG2 = testPH2[sorted2]
testLAG2 = testLAG2.reshape(len(testLAG2),-1)
testLAG2=np.transpose(testLAG2)

testPH3=np.transpose(testPH3)
testLAG3 = testPH3[sorted3]
testLAG3 = testLAG3.reshape(len(testLAG3),-1)
testLAG3=np.transpose(testLAG3)

testPH4=np.transpose(testPH4)
testLAG4 = testPH4[sorted4]
testLAG4 = testLAG4.reshape(len(testLAG4),-1)
testLAG4=np.transpose(testLAG4)

(4950, 10000)
(10000, 1000)
(4165, 10000)
(10000, 1000)
(1773, 10000)
(10000, 1000)
(385, 10000)
(10000, 385)


In [36]:

testLAG1_out = lag1.transform(testLAG1)
testLAG1_prob  = lag1.predict_proba(testLAG1)
testLAG2_out = lag2.transform(testLAG2)
testLAG2_prob = lag2.predict_proba(testLAG2)
testLAG3_out = lag3.transform(testLAG3)
testLAG3_prob = lag3.predict_proba(testLAG3)
testLAG4_out = lag4.transform(testLAG4)
testLAG4_prob = lag4.predict_proba(testLAG4)
TestFeatureMatrix = np.concatenate((testLAG1_out,testLAG2_out,testLAG3_out,testLAG4_out),axis = 1)
#save('drive/My Drive/569_Competition/FeatureMatrixForTest_ph4.npy', TestFeatureMatrix)


(10000, 50)
(10000, 50)
(10000, 50)
(10000, 50)
(10000, 200)


In [54]:

RFModel = RandomForestClassifier(n_estimators = 80, max_depth=100, random_state = 0)
RFModel.fit(FeatureMatrix_Train_, trainLabel)
print("Train a XGBoost model")
params = {"objective": "multi:softmax",
          "eta": 0.3,
          "max_depth": 10,
          "min_child_weight": 3,
          "silent": 1,
          "subsample": 0.7,
          "colsample_bytree": 0.7,
          "seed": 1,
          "num_class": 10}
gbm = xgb.train(params, xgb.DMatrix(
    FeatureMatrix_Train_, trainLabel), 250)

test_probs = gbm.predict(xgb.DMatrix(TestFeatureMatrix))
#y_pred = RFModel.predict(TestFeatureMatrix)
score  = accuracy_score(testLabel,test_probs)
print("Test Accuracy is: ", score)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Train a XGBoost model
Make predictions on the test set
0.6456


array([1, 1, 3, ..., 2, 1, 3])